In [ ]:
import numpy as np
import math
import random
import plotly.graph_objects as go
BOLD = '\033[1m'
GREEN = '\033[92m'
PURPLE = '\033[95m'
CYAN = '\033[96m'
DARKCYAN = '\033[36m'
BLUE = '\033[94m'
YELLOW = '\033[93m'
RED = '\033[91m'
UNDERLINE = '\033[4m'
END = '\033[0m'

In [ ]:
class BoyanEnvironment:

    def __init__(self, env_info):

        self.num_states = env_info['num_states']
        self.terminal_reward = -2
        self.transition_reward = -3

    def transition(self, current_state, action):

        reward = self.transition_reward
        next_state = min(13, current_state + action)
        terminal = False

        if next_state == 13:
            terminal = True
            if current_state == 12:
                reward = self.terminal_reward

        return (reward, next_state, terminal)

    

In [ ]:
def get_features(state,num_features=4):

    shape = 4
    features = np.zeros(shape)
    
    if state == 13:
        return features
    state=state-1
    features[int(state/4)] = 1 - 0.25*(state%4)
    if int(state/4 + 1) < num_features:
        features[int(state/4 + 1)] = 0.25*(state%4)

    return features

def RMS_error(y1, y2):
    return np.sqrt(np.sum((y1-y2)**2)/y1.shape[0])

def radius(mat):
    eig_val, vec = np.linalg.eig(mat)
    return max(abs(eig_val))

In [ ]:
def compute_iterative(method,lambdas=None,num_episodes=50):
    num_states = 13
    num_features = 4
    if lambdas is None:
        lambdas = [0,0.5,1]
    num_lambdas = len(lambdas)
    epsilon=1e-5
    gamma=0.99
    env = BoyanEnvironment({'num_states': num_states})

    true_y=np.zeros((13,),dtype=np.float)
    true_y[12]=0
    true_y[11]=-2
    for i in range(10,-1,-1):
        true_y[i]=0.5*(-3+gamma*true_y[i+1])+0.5*(-3+gamma*true_y[i+2])
    # print(true_y)
    total_radius=[]
    for l in range(num_lambdas):
        lamb = lambdas[l]
        print(BLUE+f"lambda={lamb}"+END)
        A = np.zeros((num_features, num_features),dtype=np.float)
        b = np.zeros(num_features,dtype=np.float)
        D = np.zeros((num_features, num_features),dtype=np.float)
        T = 1
        theta = np.random.rand(num_features)
        delta=np.zeros(num_features)
        v=np.zeros(num_features)
        error=[]
        curr_radius=[]
        for j in range(num_episodes):
            current_state = 1
            z = np.zeros_like(get_features(current_state),dtype=np.float)
            terminal = False
            while(not terminal):
                action = 1
                if current_state < 12:
                    action = random.choice([1,2])
                reward, next_state, terminal = env.transition(current_state, action)
                z = gamma*lamb*z + get_features(current_state)
                A += (np.outer(z, gamma*get_features(next_state) - get_features(current_state)) - A)/T
                b += (z*reward - b)/T
                D += (np.outer(get_features(current_state), get_features(current_state)) - D)/T
                if method == "LSTD":
                    B = A
                    
                elif method == "iLSTD":
                    B = -np.eye(num_features)
                
                elif method == "LSPE":
                    B = -D
                    
                elif method == "GRAD":
                    B = 0
                    
                else:
                    print("NO METHOD")
                    break
                e = np.dot(A, theta) + b
                if method=="GRAD":
                    delta=np.dot(A,e)
                    alpha_num = np.dot(A, np.dot(A, delta))+1e-5
                    alpha = (np.dot(np.transpose(delta), alpha_num))/np.dot(np.transpose(alpha_num), alpha_num)
                    grad=alpha*delta
                    theta -= grad
                    C=np.eye(num_features)
                else:
                    k=e-np.dot(B,delta)+epsilon
                    
                    temp=np.dot(B,k)
                    beta=-(np.dot(np.transpose(k),temp))/(np.dot(np.transpose(temp),temp))
                    delta -=beta*k
                    eps=np.dot(A,delta)
                    x=eps-np.dot(B,v)+epsilon
                
                    temp=np.dot(B,x)
                    eta=-(np.dot(np.transpose(x),temp))/(np.dot(np.transpose(temp),temp))
                    v-=eta*x
                    alpha=np.dot(delta,v)/np.dot(np.transpose(v),v)
                    theta -=alpha*delta
                    C=np.transpose(A)@B
                current_state = next_state
                T += 1
            curr_radius.append(radius(np.eye(num_features)-np.linalg.pinv(C)@np.transpose(A)@A))
            y = np.array([np.dot(theta, get_features(i)) for i in range(1,14)])
            rms_error = RMS_error(y, true_y)
            error.append(rms_error)
            if rms_error<0.01:
                break
            # error.append(rms_error)
        y = np.array([np.dot(theta, get_features(i)) for i in range(1,14)])
        rms_error=RMS_error(y,true_y)
        print(PURPLE+f"y_true={true_y}"+END)
        print(PURPLE+f"y_pred={y}"+END)
        print(GREEN+f"w={theta}"+END)
        print(RED+f"rms_error={rms_error}"+END)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=[i for i in range(1,len(error)+1)],y=error,name=f"method,λ={lamb}"))
        fig.update_layout(title='MSE vs #Episodes',
                   xaxis_title='#Episode',
                   yaxis_title='MSE Loss')
        fig.show()
        total_radius.append(curr_radius)
    return total_radius

In [ ]:
methods=["LSTD","iLSTD","LSPE","GRAD"]
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
all_methods_radius=[]
lambdas=[0.6,0.8]
for method in methods:
    print(BOLD+DARKCYAN+f"Method={method}"+END)
    all_methods_radius.append(compute_iterative(method,lambdas,500))

x = [i+1 for i in range(1,100)]
for j in range(len(lambdas)):
    print(BLUE+f"lambda={lambdas[j]}"+END)
    fig = go.Figure()
    for i in range(4):
        curr_method=all_methods_radius[i]
        fig.add_trace(go.Scatter(x=x,y=curr_method[j],name=methods[i]))
    fig.update_layout(title='Spectral radius vs #Episodes',
                   xaxis_title='#Episode',
                   yaxis_title='Spectral Radius')
    fig.show()

Method=LSTD
lambda=0.6
y_true=[-23.16 -21.30 -19.43 -17.55 -15.65 -13.74 -11.81 -9.88 -7.93 -5.97 -3.99
 -2.00 0.00]
y_pred=[-23.22 -21.32 -19.41 -17.51 -15.60 -13.70 -11.80 -9.90 -7.99 -6.00 -4.00
 -2.01 0.00]
w=[-23.22 -15.60 -7.99 -0.01]
rms_error=0.03375514982049806


lambda=0.8
y_true=[-23.16 -21.30 -19.43 -17.55 -15.65 -13.74 -11.81 -9.88 -7.93 -5.97 -3.99
 -2.00 0.00]
y_pred=[-23.12 -21.26 -19.41 -17.55 -15.69 -13.77 -11.85 -9.93 -8.01 -6.01 -4.01
 -2.00 0.00]
w=[-23.12 -15.69 -8.01 -0.00]
rms_error=0.03868593863223034


Method=iLSTD
lambda=0.6
y_true=[-23.16 -21.30 -19.43 -17.55 -15.65 -13.74 -11.81 -9.88 -7.93 -5.97 -3.99
 -2.00 0.00]
y_pred=[-23.11 -21.24 -19.36 -17.48 -15.60 -13.65 -11.70 -9.75 -7.79 -5.86 -3.93
 -2.00 0.00]
w=[-23.11 -15.60 -7.79 -0.06]
rms_error=0.08215242116353129


lambda=0.8
y_true=[-23.16 -21.30 -19.43 -17.55 -15.65 -13.74 -11.81 -9.88 -7.93 -5.97 -3.99
 -2.00 0.00]
y_pred=[-23.07 -21.19 -19.31 -17.43 -15.55 -13.62 -11.69 -9.76 -7.83 -5.88 -3.93
 -1.97 0.00]
w=[-23.07 -15.55 -7.83 -0.02]
rms_error=0.09843608397549419


Method=LSPE
lambda=0.6
y_true=[-23.16 -21.30 -19.43 -17.55 -15.65 -13.74 -11.81 -9.88 -7.93 -5.97 -3.99
 -2.00 0.00]
y_pred=[-23.34 -21.44 -19.55 -17.65 -15.76 -13.83 -11.91 -9.98 -8.06 -6.04 -4.03
 -2.02 0.00]
w=[-23.34 -15.76 -8.06 -0.00]
rms_error=0.10464465911943883


lambda=0.8
y_true=[-23.16 -21.30 -19.43 -17.55 -15.65 -13.74 -11.81 -9.88 -7.93 -5.97 -3.99
 -2.00 0.00]
y_pred=[-23.08 -21.22 -19.37 -17.51 -15.66 -13.74 -11.82 -9.90 -7.99 -5.99 -3.99
 -1.99 0.00]
w=[-23.08 -15.66 -7.99 0.01]
rms_error=0.041945811693331524


Method=GRAD
lambda=0.6
y_true=[-23.16 -21.30 -19.43 -17.55 -15.65 -13.74 -11.81 -9.88 -7.93 -5.97 -3.99
 -2.00 0.00]
y_pred=[-23.45 -21.51 -19.58 -17.64 -15.70 -13.80 -11.90 -10.00 -8.10 -6.08 -4.07
 -2.05 0.00]
w=[-23.45 -15.70 -8.10 -0.03]
rms_error=0.136528428029866


lambda=0.8
y_true=[-23.16 -21.30 -19.43 -17.55 -15.65 -13.74 -11.81 -9.88 -7.93 -5.97 -3.99
 -2.00 0.00]
y_pred=[-23.24 -21.36 -19.48 -17.60 -15.72 -13.80 -11.87 -9.95 -8.02 -6.03 -4.04
 -2.06 0.00]
w=[-23.24 -15.72 -8.02 -0.07]
rms_error=0.06288349541644686


lambda=0.6


lambda=0.8
